In [35]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

import data_preparing
import forward

In [36]:
mean1 = [1.0]
mean2 = [2.0]
var1 = [1.0]
var2 = [2.0]
p = 0.1
M = 100
betas = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [37]:
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean, var) ??? 
])

gmm_data = data_preparing.create_dataset(M, mean1, mean2, var1, var2, p)

train_data, test_data = train_test_split(gmm_data, test_size=0.25)
train_data = transform(train_data)
test_data = transform(test_data)

In [38]:
batch_size = 25
train_loader = DataLoader(train_data, batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size, shuffle=False)

In [39]:
class RevNet(nn.Module):
    def __init__(self, N):
        super().__init__()
        self.common_l1 = nn.Sequential(
            nn.Linear(N, N * 2),
            nn.ReLU(),
            nn.BatchNorm1d(N * 2),
        )
        self.common_l2 = nn.Sequential(
            nn.Linear(N * 2, N * 4),
            nn.ReLU(),
            nn.BatchNorm1d(N * 4),
        )
        self.mean = nn.Linear(N * 4, N)
        self.log_var = nn.Linear(N * 4, N)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.common_l1(x)
        x = self.common_l2(x)
        return self.mean(x), self.log_var(x)